<a href="https://colab.research.google.com/github/suwisitlk/229352-StatisticalLearning/blob/main/Lab03_kNN_using_Grid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #3

In [2]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


In [3]:
print("X[0]:", Xtrain[0])
print("y[0]:", ytrain[0])

X[0]: From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





y[0]: 7


In [4]:
train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Apply Tfidf ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html))

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# define transformation
tfidf = TfidfVectorizer(stop_words="english", ngram_range=(1,1)) #stop word คือการเอาคำที่ไม่สำคัญออกในภาาาอังกฤษ  ngram_range คือจำนวนแกรมต่อหนึ่งคำ เช่น black cat


# fit+transform training set
Xtrain_tfidf = tfidf.fit_transform(Xtrain)
Xtest_tfidf = tfidf.transform(Xtest)


# See output
Xtrain_tfidf .shape

(3000, 61689)

In [9]:
# tfidf.vocabulary_

#### Exercise 1: Find post in the training set that is closest in tf-idf to the first post in the test set (`Xtest[0]`). Print the content of both posts (not the tf-idf vectors).

### Classify with k-Nearest Neighbor (kNN) ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html))

In [24]:
from sklearn.neighbors import KNeighborsClassifier

# Define model
nb = KNeighborsClassifier(n_neighbors=5)

# Fit the model
nb.fit(Xtrain_tfidf, ytrain)

KNeighborsClassifier()

Evaluate on the test set using [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

We will focus on the [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [25]:
from sklearn.metrics import classification_report
import numpy as np

# Make predictions on the test set
ypred = nb.predict(Xtest_tfidf)

np.array(train.target_names)[ypred]

array(['talk.religion.misc', 'talk.politics.guns', 'talk.religion.misc',
       'talk.politics.mideast', 'alt.atheism', 'sci.med',
       'soc.religion.christian', 'alt.atheism', 'comp.windows.x',
       'comp.graphics', 'sci.electronics', 'comp.windows.x',
       'talk.politics.mideast', 'rec.motorcycles',
       'soc.religion.christian', 'comp.sys.ibm.pc.hardware',
       'comp.os.ms-windows.misc', 'rec.motorcycles', 'misc.forsale',
       'rec.autos', 'talk.politics.mideast', 'misc.forsale',
       'soc.religion.christian', 'sci.space', 'comp.sys.ibm.pc.hardware',
       'misc.forsale', 'sci.crypt', 'misc.forsale', 'rec.autos',
       'sci.space', 'comp.graphics', 'comp.windows.x',
       'comp.sys.ibm.pc.hardware', 'comp.sys.ibm.pc.hardware',
       'sci.space', 'comp.graphics', 'rec.sport.baseball', 'rec.autos',
       'comp.graphics', 'comp.graphics', 'talk.politics.mideast',
       'rec.autos', 'rec.motorcycles', 'comp.graphics', 'sci.crypt',
       'rec.sport.baseball', 'comp.w

In [21]:
#report classification scores
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.35      0.43      0.38        21
           1       0.33      0.48      0.39        21
           2       0.46      0.62      0.52        26
           3       0.45      0.56      0.50        34
           4       0.62      0.47      0.53        34
           5       0.88      0.54      0.67        26
           6       0.35      0.55      0.43        22
           7       0.53      0.57      0.55        28
           8       0.80      0.73      0.76        33
           9       0.77      0.68      0.72        25
          10       0.67      0.74      0.70        27
          11       0.76      0.95      0.84        20
          12       0.36      0.21      0.26        24
          13       0.78      0.61      0.68        23
          14       0.84      0.75      0.79        28
          15       0.62      0.62      0.62        29
          16       0.56      0.71      0.62        21
          17       0.82    

### Combine all methods into a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

* การรวมฟังชั่น tfidf กับ knn

In [26]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([ ('tfidf', TfidfVectorizer(),
                      ('nb', KNeighborsClassifier())
)])

# Fit the pipeline to the training set
pipeline.fit(Xtrain, ytrain)

# Make predictions on the test set
ypred = pipeline.predict(Xtest)

# report classification scores
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.18      0.48      0.26        21
           1       0.38      0.48      0.43        21
           2       0.56      0.58      0.57        26
           3       0.52      0.50      0.51        34
           4       0.67      0.53      0.59        34
           5       0.88      0.54      0.67        26
           6       0.50      0.41      0.45        22
           7       0.58      0.54      0.56        28
           8       0.74      0.70      0.72        33
           9       0.82      0.72      0.77        25
          10       0.75      0.56      0.64        27
          11       0.71      0.85      0.77        20
          12       0.31      0.21      0.25        24
          13       0.79      0.48      0.59        23
          14       0.76      0.57      0.65        28
          15       0.67      0.55      0.60        29
          16       0.56      0.67      0.61        21
          17       0.59    

Now we will use [grid search cross-validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find model with the best hyperparameters

![5CV](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In [31]:
from sklearn.model_selection import GridSearchCV

params = {'tfidf__ngram_range': [(1, 1), (2, 2)],
          'nb__n_neighbors': [1, 3, 5]}

# Define GridSearchCV
gridcv = GridSearchCV(pipeline, params,
                      scoring='f1_macro', cv=5) #cv คือจำนวนเซ็ตที่เราแบ่ง

# Fit and cross-validate the model on 3-fold data
gridcv.fit(Xtrain, ytrain)

# Print the best parameters
gridcv.best_params_

{'nb__n_neighbors': 1, 'tfidf__ngram_range': (1, 1)}

In [28]:
gridcv.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('nb', KNeighborsClassifier(n_neighbors=1))])

In [29]:
# Make predictions on the test set
ypred = gridcv.predict(Xtest)

# Report classification scores
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.29      0.52      0.37        21
           1       0.38      0.43      0.40        21
           2       0.54      0.54      0.54        26
           3       0.46      0.38      0.42        34
           4       0.81      0.50      0.62        34
           5       0.68      0.50      0.58        26
           6       0.41      0.41      0.41        22
           7       0.67      0.57      0.62        28
           8       0.82      0.70      0.75        33
           9       0.69      0.72      0.71        25
          10       0.68      0.56      0.61        27
          11       0.70      0.80      0.74        20
          12       0.43      0.42      0.43        24
          13       0.65      0.57      0.60        23
          14       0.64      0.64      0.64        28
          15       0.48      0.52      0.50        29
          16       0.63      0.57      0.60        21
          17       0.43    

#### Exercise 2:

1. Use grid search 5-fold cross-validation across different values of the following two kNN parameters: `n_neighbors` and `metric`  **on the training set** to find the best model.

2. For the best value of `n_neighbors` and `metric` you found above, compute the `f1_macro` score **on the test set**.
* Print the value of `n_neighbors` and `metric`.
* Print the model's `f1_macro` score.

In [34]:
params = {'tfidf__ngram_range': [(1, 1), (2, 2)],
          'nb__n_neighbors': [1, 3, 5, 7, 9 ,11 ],
          'nb__weights': ['uniform', 'distance'], # ลองให้น้ำหนักเท่ากันและให้น้ำหนักตามระยะทาง
          'nb__metric': ['euclidean', 'manhattan', 'cosine']
}

# Define GridSearchCV
gridcv = GridSearchCV(pipeline, params,
                      scoring='f1_macro', cv=5,n_jobs=-1) #cv คือจำนวนเซ็ตที่เราแบ่ง

# Fit and cross-validate the model on 55-fold data
gridcv.fit(Xtrain, ytrain)

# Print the best parameters
gridcv.best_params_

{'nb__metric': 'cosine',
 'nb__n_neighbors': 3,
 'nb__weights': 'distance',
 'tfidf__ngram_range': (1, 1)}

In [35]:
# Make predictions on the test set
ypred = gridcv.predict(Xtest)

# Report classification scores
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.29      0.57      0.39        21
           1       0.44      0.38      0.41        21
           2       0.50      0.54      0.52        26
           3       0.52      0.44      0.48        34
           4       0.78      0.53      0.63        34
           5       0.75      0.58      0.65        26
           6       0.50      0.45      0.48        22
           7       0.68      0.68      0.68        28
           8       0.74      0.70      0.72        33
           9       0.70      0.76      0.73        25
          10       0.67      0.52      0.58        27
          11       0.77      0.85      0.81        20
          12       0.42      0.33      0.37        24
          13       0.65      0.57      0.60        23
          14       0.67      0.64      0.65        28
          15       0.58      0.48      0.53        29
          16       0.58      0.52      0.55        21
          17       0.48    